# Deploy CodeLlama-7b on Amazon SageMaker using the Hugging Face TGI Container.

Code Llama is a set of generative text models, pretrained and fine-tuned, with sizes ranging from 7 billion to 34 billion parameters. The 7B instruct-tuned version, available in Hugging Face Transformers format, is designed for tasks related to code synthesis and understanding.

[Hugging Face Text Generation Inference (TGI)](https://huggingface.co/docs/text-generation-inference/en/index) is a specialized environment designed for deploying large language models (LLMs) efficiently for text generation tasks. It provides an optimized, scalable solution for serving models like GPT, Code Llama, or other large transformer-based models, especially when deployed in production environments. TGI enables high-performance text generation using Tensor Parallelism and dynamic batching for the most popular open-source LLMs, including StarCoder, BLOOM, GPT-NeoX, Llama, and T5.

In this notebook, you will learn how to deploy and speed up [Neuronx model for CodeLlama](https://huggingface.co/aws-neuron/CodeLlama-7b-hf-neuron-8xlarge) inference using AWS Inferentia2 on Amazon SageMaker. 

## License agreement
View license information https://huggingface.co/meta-llama before using the model.
This notebook is a sample notebook and not intended for production use. Please refer to the licence at https://github.com/aws/mit-0.

## Intro to AWS Inferentia 2

[AWS inferentia (Inf2)](https://aws.amazon.com/de/ec2/instance-types/inf2/) are purpose-built EC2 for deep learning (DL) inference workloads. Inferentia 2 is the successor of [AWS Inferentia](https://aws.amazon.com/ec2/instance-types/inf1/?nc1=h_ls), which promises to deliver up to 4x higher throughput and up to 10x lower latency.

| instance size | accelerators | Neuron Cores | accelerator memory | vCPU | CPU Memory | on-demand price ($/h) |
| ------------- | ------------ | ------------ | ------------------ | ---- | ---------- | --------------------- |
| inf2.xlarge   | 1            | 2            | 32                 | 4    | 16         | 0.76                  |
| inf2.8xlarge  | 1            | 2            | 32                 | 32   | 128        | 1.97                  |
| inf2.24xlarge | 6            | 12           | 192                | 96   | 384        | 6.49                  |
| inf2.48xlarge | 12           | 24           | 384                | 192  | 768        | 12.98                 |

Additionally, inferentia 2 will support the writing of custom operators in c++ and new datatypes, including `FP8` (cFP8).

## Step 1: Import Libraries

We are going to use the [Neuronx model for codellama/CodeLlama-7b-hf](https://huggingface.co/aws-neuron/CodeLlama-7b-hf-neuron-8xlarge).

In [ ]:
import json
import sagemaker
import boto3
from sagemaker.huggingface import HuggingFaceModel, get_huggingface_llm_image_uri

- **json**: A built-in Python library for working with JSON data.
- **sagemaker**: The Amazon SageMaker Python SDK used to interact with SageMaker.
- **boto3**: AWS SDK for Python, used to interact with AWS services (here, it is used to get the IAM role if it's not available).
- **HuggingFaceModel**: A SageMaker class that simplifies the deployment of Hugging Face models to SageMaker.
- **get_huggingface_llm_image_uri**: A utility function to retrieve the proper Docker image URI for the Hugging Face large language models (LLMs).

## Step 2: Define the SageMaker Execution Role

In [ ]:
try:
	role = sagemaker.get_execution_role()
except ValueError:
	iam = boto3.client('iam')
	role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

- The **role** is the AWS Identity and Access Management (IAM) role that SageMaker uses to access resources like S3 and EC2.
- The code attempts to fetch the SageMaker execution role using `sagemaker.get_execution_role()`. If this fails (such as when running the script outside of SageMaker Studio or a notebook), it falls back to using `boto3` to retrieve the IAM role directly by name (`'sagemaker_execution_role'`).

## Step 3: Define Model Configuration


In [ ]:
# Hub Model configuration. https://huggingface.co/models
hub = {
    "HF_MODEL_ID": "codellama/CodeLlama-7b-Instruct-hf",
    "HF_NUM_CORES": "2",
    "HF_AUTO_CAST_TYPE": "fp16",
    "MAX_BATCH_SIZE": "4",
    "MAX_INPUT_TOKENS": "3686",
    "MAX_TOTAL_TOKENS": "4096",
}

  - **hub**: A dictionary containing the model configuration for Hugging Face.
  - **'HF_MODEL_ID'**: The ID of the Hugging Face model to deploy. Here, it is `codellama/CodeLlama-7b-Instruct-hf`, which is the instruct-tuned version of the 7-billion parameter Code Llama model.
  - **'HF_NUM_CORES'**: This parameter defines the number of Neuron Cores that will be used for inference. AWS Inferentia processors are divided into multiple cores, and by setting this value to "2", you are instructing the model to use two cores during inference, which can improve performance by enabling parallelism.
  - **'HF_AUTO_CAST_TYPE'**: This setting enables automatic mixed precision for inference by specifying fp16 (16-bit floating-point) as the type of computation. Using mixed precision (fp16 instead of the default fp32) can significantly reduce memory usage and speed up inference without sacrificing much accuracy. This is especially useful when running large models like Code Llama on hardware with limited memory.
  - **'MAX_BATCH_SIZE'**: Batch size determines how many inputs (or requests) the model will process at once. A batch size of 4 means that the model will process four inputs simultaneously during each forward pass, which can optimize throughput and make better use of hardware resources like GPU or Inferentia cores.
  - **'MAX_INPUT_TOKENS'**: This setting defines the maximum number of input tokens that the model can process in a single inference request.
  - **'MAX_TOTAL_TOKENS'**: This setting defines the maximum number of tokens (input + output) that the model will handle in total during an inference request.

## Step 4: Create the Hugging Face Model

In [ ]:
# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
    image_uri=get_huggingface_llm_image_uri("huggingface-neuronx", version="0.0.24"),
    env=hub,
    role=role,
)

- **huggingface_model**: This variable stores an instance of the `HuggingFaceModel` class, which handles the deployment.
- **image_uri**: This specifies the container image for the Hugging Face LLM that will be used for inference. The `get_huggingface_llm_image_uri` function automatically retrieves the correct image URI based on the framework (Hugging Face) and version (`2.2.0`).
- **env**: Specifies the environment variables needed for the model, such as the model ID and the number of GPUs.
- **role**: This is the SageMaker execution role that allows SageMaker to access AWS resources.

## Step 5: Deploy the Model to SageMaker

Ignore the warning that model is not compiled. 

In [ ]:
# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
    initial_instance_count=1,
    instance_type="ml.inf2.xlarge",
    container_startup_health_check_timeout=3600,
    volume_size=512,
)

- **predictor**: This variable stores the deployed model instance.
- **initial_instance_count**: The number of instances for serving the model. Here, only one instance is deployed (`1`).
- **instance_type**: The type of instance to deploy the model on. In this case, it's `ml.inf2.8xlarge`, which is an instance type optimized for inference on machine learning models.
- **container_startup_health_check_timeout**: This sets the health check timeout (in seconds) for the container startup. A timeout of 300 seconds (5 minutes) is given to allow the large model to initialize properly.


## Step 6: Make an Inference Request
The model will process this input and return a response, generating Python code that prints "Hello, World".

In [ ]:
# send request
res = predictor.predict(
    {
        "inputs": "Write an example python program?",
        "parameters": {
            "do_sample": True,
            "max_new_tokens": 128,
            "temperature": 0.7,
            "top_k": 50,
            "top_p": 0.95,
        }
    }
)

In [ ]:
print(res[0]['generated_text'])

- **predictor.predict()**: This sends a request to the deployed model's endpoint for inference.
- **inputs**: The input provided to the model for inference. In this case, it is a simple request asking the model to "Generate python code for Hello World program".

## Step 7: Clean up the environment

In [ ]:
sm_client.delete_endpoint(EndpointName=endpoint_name)
sm_client.delete_endpoint_config(EndpointConfigName=endpoint_config_name)
sm_client.delete_model(ModelName=model_name)

predictor.delete_model()
predictor.delete_endpoint()

#### Additional Resources:
- [Neuron Samples](https://github.com/aws-neuron/aws-neuron-samples)